In [ ]:
import torch
import torch.nn as nn
import pandas as pd

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

from collections import Counter

from typing import Dict, List

nltk.download('wordnet')

# Choose device

In [2]:
device: str = "cpu"

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"

# Dataset

In [17]:
data = pd.read_csv("../data/IMDB.csv")

data = pd.get_dummies(data, columns=["sentiment"])
data = data.rename({"review": "sentence", "sentiment_negative": "negative", "sentiment_positive": "positive"}, axis=1)

In [18]:
X = data[["sentence"]]
y = data.drop(["sentence"], axis=1)

In [26]:
sentences = X["sentence"].values
words = " ".join(sentences)
words = words.split()

In [133]:
class Tokenizer:
    def __init__(self, words: List[str]) -> None:
        counter = Counter(words)
        self.vocab = sorted(counter, key=counter.get, reverse=True)
        self.int2word = dict(enumerate(self.vocab, 1))
        self.int2word[0] = "<PAD>"
        self.word2int = {word: id for id, word in self.int2word.items()}
        
    def __len__(self) -> int:
        return len(self.vocab)
    
    def tokenize(self, sentence: str) -> List[str]:
        return sentence.split()
    
    def convert_tokens_to_ids(self, tokens: List[str]) -> List[int]:
        return [self.word2int[word] for word in tokens]
    
    def convert_ids_to_tokens(self, ids: List[int] | torch.LongTensor) -> List[str]:
        if isinstance(ids, torch.LongTensor):
            return [self.int2word[id.item()] for id in ids]
        else: 
            return [self.int2word[id] for id in ids]
    
tokenizer = Tokenizer(words)
        

In [137]:
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

class IMDBDataset(Dataset):
    def __init__(self, root: str = "../data/IMDB.csv", train: bool = True, test_size: float = 0.33, tokenizer: Tokenizer = None) -> None:
        super().__init__()

        data = pd.read_csv(root)
        data = pd.get_dummies(data, columns=["sentiment"])
        data = data.rename({"review": "sentence", 
                            "sentiment_negative": "negative", 
                            "sentiment_positive": "positive"}, axis=1)
        
        X = data["sentence"].values
        y = data.drop(["sentence"], axis=1).values

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

        if train:
            self.X, self.y = X_train, y_train
        else:
            self.X, self.y = X_test, y_test

        self.tokenizer: Tokenizer = tokenizer

    def __len__(self) -> int:
        return len(self.X)
    
    def __getitem__(self, idx):
        sentence = self.X[idx]
        tokens = self.tokenizer.tokenize(sentence)
        ids = self.tokenizer.convert_tokens_to_ids(tokens)

        ids = torch.tensor(ids, dtype=torch.long)
        labels = torch.tensor(self.y[idx], dtype=torch.float32)
        return ids, labels


In [144]:
from torch.utils.data import DataLoader

train_dataset = IMDBDataset(tokenizer=tokenizer)
test_dataset = IMDBDataset(train=False, tokenizer=tokenizer)

batch_size: int = 32

train_loader = DataLoader(train_dataset)
test_loader = DataLoader(test_dataset)

# Model

In [145]:
class EmotionalLSTM(nn.Module):
    def __init__(self, 
                 vocab_size: int, 
                 emb_size: int, 
                 hidden_size: int, 
                 num_stacked_layers: int = 3, 
                 dropout: float = 0.2) -> None:
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.lstm = nn.LSTM(emb_size, hidden_size, num_stacked_layers, dropout=dropout, batch_first=True)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(hidden_size, 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x: torch.LongTensor) -> torch.Tensor:
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = out[:,-1,:]
        out = self.dropout(out)
        out = self.fc(out)
        out = self.sigmoid(out)

        return out

vocab_size: int = 10000 # change
emb_size: int = 256
hidden_size: int = 512
num_stacked_layers: int = 2
dropout: float = 0.25

model: EmotionalLSTM = EmotionalLSTM(vocab_size, emb_size, hidden_size, num_stacked_layers, dropout).to(device)

# Train

In [ ]:
from tqdm import tqdm
import torch.optim as optim

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

train_loss_history = []
test_loss_history = []

for epoch in range(num_epochs):
    train_loss = 0.0
    test_loss = 0.0


    model.train()
    for sentences, labels in tqdm(train_loader):
        sentences, labels = sentences.to("mps"), labels.to("mps")

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(sentences)

        # Calculate loss
        loss = criterion(outputs, labels)

        train_loss_history.append(loss.item())

        # Backward pass and optimize
        loss.backward()
        optimizer.step()
    train_loss = loss.item()

    model.eval()
    for sentences, labels in test_loader:
        sentectes, labels = sentences.to("mps"), labels.to("mps")

        with torch.no_grad():
            outputs = model(sentectes)

        loss = criterion(outputs, labels)
        test_loss_history.append(loss.item())

    test_loss = loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Val_loss: {test_loss:.4f}')


In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss_history, label="train_loss")
#plt.plot(test_loss_history, label="val_loss")
plt.xlabel("batch")
plt.ylabel("loss")
plt.legend()
plt.show()

In [ ]:
accuracy = 0
total_samples = 0

loader = test_loader

model.eval()  # Set the model to evaluation mode
for batch, (X, y) in enumerate(loader):
    with torch.no_grad():
        y_pred = model(X.to("mps"))
    
    y = y.to("mps")
    _, predicted = torch.max(y_pred, 1)
    
    # Convert one-hot encoded y to class indices if needed
    if y.dim() == 2:
        y = torch.argmax(y, dim=1)
    
    # Debug prints for shapes
    correct_predictions = (predicted == y).sum().item()
    accuracy += correct_predictions
    total_samples += y.size(0)

accuracy / (len(loader) * loader.batch_size)